<a href="https://colab.research.google.com/github/deckel28/music-mood/blob/master/mood_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
import os
import pickle
import h5py
import numpy as np
from numpy import savetxt
from numpy import genfromtxt
from numpy import random
from random import choices
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
from tensorflow import keras
from keras import metrics
from keras import backend
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D, LSTM, Dropout, BatchNormalization
import sklearn
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import fbeta_score
from sklearn.metrics import multilabel_confusion_matrix

Using TensorFlow backend.


In [ ]:
os.chdir('/content/drive/My Drive')
os.listdir()

In [5]:
trainY = genfromtxt('Datasets/encoded_train.csv', delimiter=',')
h5f = h5py.File('Datasets/feats_train.h5','r')
trainX = h5f['dataset'][:]
h5f.close()

valY = genfromtxt('Datasets/encoded_val.csv', delimiter=',')
h5f = h5py.File('Datasets/feats_val.h5','r')
valX = h5f['dataset'][:]
h5f.close()

testY = genfromtxt('Datasets/encoded_test.csv', delimiter=',')
h5f = h5py.File('Datasets/feats_test.h5','r')
testX = h5f['dataset'][:]
h5f.close()

In [6]:
print('trainX-', trainX.shape, '   trainY- ', trainY.shape)
print('valX-', valX.shape, '      valY- ', valY.shape)
print('testX-', testX.shape, '     testY- ', testY.shape)

trainX- (2802, 20, 9762)    trainY-  (2802, 8)
valX- (346, 20, 9762)       valY-  (346, 8)
testX- (312, 20, 9762)      testY-  (312, 8)


In [8]:
totalY = np.concatenate((trainY, valY, testY))
print(np.sum(totalY, axis=0))

[3009. 1528.    9. 1052.  171.  796.   27.  419.]


In [9]:
trainX0 = np.expand_dims(trainX, axis=-1)
valX0 = np.expand_dims(valX, axis=-1)
testX0 = np.expand_dims(testX, axis=-1)

IN_SHAPE = trainX0.shape[1:]
print(IN_SHAPE)

(20, 9762, 1)


In [10]:
print(trainX0.shape, valX0.shape, testX0.shape)
print(trainY.shape, valY.shape, testY.shape)

(2802, 20, 9762, 1) (346, 20, 9762, 1) (312, 20, 9762, 1)
(2802, 8) (346, 8) (312, 8)


In [11]:
def generator(features, labels, batch_size):
    # Create empty arrays to contain batch of features and labels#
    batch_features = np.zeros((batch_size, IN_SHAPE[0], IN_SHAPE[1], 1))
    batch_labels = np.zeros((batch_size, 8))
    while True:
        for i in range(batch_size):
            index = random.choice(len(features), 1)
            batch_features[i] = features[index]
            batch_labels[i] = labels[index]
        yield batch_features, batch_labels

In [12]:
def fbeta(y_true, y_pred, beta=2):
  # clip predictions
  y_pred = backend.clip(y_pred, 0, 1)
  # calculate elements
  tp = backend.sum(backend.round(backend.clip(y_true * y_pred, 0, 1)), axis=1)
  fp = backend.sum(backend.round(backend.clip(y_pred - y_true, 0, 1)), axis=1)
  fn = backend.sum(backend.round(backend.clip(y_true - y_pred, 0, 1)), axis=1)
  # calculate precision
  p = tp / (tp + fp + backend.epsilon())
  # calculate recall
  r = tp / (tp + fn + backend.epsilon())
  # calculate fbeta, averaged across each class
  bb = beta ** 2
  fbeta_score = backend.mean((1 + bb) * (p * r) / (bb * p + r + backend.epsilon()))
  return fbeta_score

In [14]:
def define_model(in_shape, out_shape):
  model = Sequential()
  
  model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=in_shape))
  model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(BatchNormalization())
  model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Dropout(0.25))
  model.add(Flatten())
  model.add(Dense(60, activation='relu', kernel_initializer='he_uniform'))
  model.add(Dense(out_shape, activation='sigmoid'))
  return model

In [15]:
model = define_model(in_shape = IN_SHAPE, out_shape=8)
model.summary()
opt = keras.optimizers.Adam(lr=0.0001)
model.compile(optimizer=opt, loss='binary_crossentropy', 
               metrics=['accuracy', fbeta, keras.metrics.categorical_accuracy])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 20, 9762, 64)      640       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 9762, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 4881, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 4881, 64)      36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 10, 4881, 64)      256       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 10, 4881, 64)      36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 2440, 64)      

In [16]:
model.fit_generator(generator(trainX0, trainY, 24),
                    verbose=1,
                    steps_per_epoch=120,
                    epochs=5,
                    validation_data=(valX0, valY))

Epoch 1/5
120/120 [==============================] - 198s 2s/step - loss: 1.7730 - accuracy: 0.8124 - fbeta: 0.6258 - categorical_accuracy: 0.7010 - val_loss: 0.7492 - val_accuracy: 0.8555 - val_fbeta: 0.6344 - val_categorical_accuracy: 0.8584
Epoch 2/5
120/120 [==============================] - 186s 2s/step - loss: 0.4032 - accuracy: 0.8587 - fbeta: 0.6882 - categorical_accuracy: 0.7819 - val_loss: 0.4047 - val_accuracy: 0.8461 - val_fbeta: 0.6590 - val_categorical_accuracy: 0.7977
Epoch 3/5
120/120 [==============================] - 186s 2s/step - loss: 0.3126 - accuracy: 0.8841 - fbeta: 0.7343 - categorical_accuracy: 0.8059 - val_loss: 0.4445 - val_accuracy: 0.7988 - val_fbeta: 0.6577 - val_categorical_accuracy: 0.2948
Epoch 4/5
120/120 [==============================] - 186s 2s/step - loss: 0.2610 - accuracy: 0.8986 - fbeta: 0.7761 - categorical_accuracy: 0.8271 - val_loss: 0.3688 - val_accuracy: 0.8555 - val_fbeta: 0.6558 - val_categorical_accuracy: 0.8642
Epoch 5/5
120/120 [=====

In [ ]:
model.save('Models/modelxxx.h5')

In [13]:
# train_yhat = np.asarray([np.ones(data_Y.shape[1]) for _ in range(data_Y.shape[0])])
# # test_yhat = np.asarray([np.ones(data_Y[180000:].shape[1]) for _ in range(data_Y[180000:].shape[0])])
# print(train_yhat.shape, data_Y.shape)
# # evaluate predictions with sklearn
# train_score = fbeta_score(data_Y, train_yhat, 2, average='samples')
# # test_score = fbeta_score(data_Y[180000:], test_yhat, 2, average='samples')
# # print('All Ones (sklearn): train=%.3f, test=%.3f' % (train_score, test_score))
# print(train_score)
# # evaluate predictions with keras
# train_score = fbeta(backend.variable(data_Y), backend.variable(train_yhat))
# # test_score = fbeta(backend.variable(data_Y[180000:]), backend.variable(test_yhat))
# # print('All Ones (keras): train=%.3f, test=%.3f' % (train_score, test_score))
# print(train_score)
# from sklearn.metrics import accuracy_score
# train_score = accuracy_score(data_Y, train_yhat)
# print(train_score)
# print(keras.metrics.accuracy(data_Y, train_yhat))